In [14]:
import pandas as pd
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer

df=pd.read_csv('C:\\temp\\2020_July_items_master.csv')

Part_Desc='PART_DESC'

df['PART NO_DESC']=df[Part_Desc].str.strip()
df['PART NO_DESC']=df[Part_Desc].str.replace('-',",", regex=True)
df['PART NO_DESC']=df[Part_Desc].str.upper()

df.rename(columns={'Items_': 'items_'},inplace=True)
df.sort_values('items_')

,PART_NO,PART_DESC,items_,PART_NO 12NC,PART NO_DESC
616,77-606-0211401-02,Assy. ACC Emitter High Numerical Aperture High...,ACC Emitter,77-606-0211401,ASSY. ACC EMITTER HIGH NUMERICAL APERTURE HIGH...
617,77-606-0211401-03,Assy. ACC Emitter High Numerical Aperture High...,ACC Emitter,77-606-0211401,ASSY. ACC EMITTER HIGH NUMERICAL APERTURE HIGH...
618,77-606-0211404-00,Assy. ACC Emitter High Numerical Aperture High...,ACC Emitter,77-606-0211404,ASSY. ACC EMITTER HIGH NUMERICAL APERTURE HIGH...
619,77-606-0211404-02,Assy. ACC Emitter High Numerical Aperture High...,ACC Emitter,77-606-0211404,ASSY. ACC EMITTER HIGH NUMERICAL APERTURE HIGH...
620,77-606-0211405-00,"Assy. ACC Emitter, w/ Power Level 4",ACC Emitter,77-606-0211405,"ASSY. ACC EMITTER, W/ POWER LEVEL 4"
...,...,...,...,...,...
300,77-594-X162561-00,Krytox 16256 vacuum oil,other,77-594-X162561,KRYTOX 16256 VACUUM OIL
301,77-601-0200607-00,eScan500 to eScan600 Upgrade Package,other,77-601-0200607,ESCAN500 TO ESCAN600 UPGRADE PACKAGE
303,77-601-1701301-02,Assy. Power Conditioner including PDS,other,77-601-1701301,ASSY. POWER CONDITIONER INCLUDING PDS
1147,77-800-810972-000,"""Cable Tie, 12""""L, Red""",other,77-800-810972-,"""CABLE TIE, 12""""L, RED"""


In [4]:
from sklearn.model_selection import train_test_split
desc=df['PART NO_DESC'].values
item=df['items_'].values
Xd_train, Xd_test, y_train, y_test= train_test_split(desc, item, test_size=0.3)


In [5]:
vectorizer=CountVectorizer(min_df=0.0001,   strip_accents='unicode',)
X_train=vectorizer.fit_transform(Xd_train)
X_test=vectorizer.transform(Xd_test)
X_train

<803x1486 sparse matrix of type '<class 'numpy.int64'>'
	with 5419 stored elements in Compressed Sparse Row format>

# Base model

In [6]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
score

C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.927536231884058

In [7]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)
print (score)

C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9391304347826087


In [8]:
from sklearn.svm import SVC
this_C=1
this_gamma=0.01
clf2=SVC(kernel='linear',C=this_C, gamma=this_gamma).fit(X_train, y_train)
score=clf2.score(X_test, y_test)

print (score)

# print (y_test)
predict=clf2.predict(X_test)

t_result=pd.DataFrame({'y_test':y_test, 'predict':predict})
t_result.to_csv('c:\\temp\\t_result2.csv')

0.9507246376811594


# from sklearn.tree import DecisionTreeClassifier
import pydotplus
clf=DecisionTreeClassifier()
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)
print (score)


# predict all data to trace back the in accuracy

In [9]:
Xd_all=desc
y_all=item
X_all=vectorizer.transform(Xd_all)
score=clf2.score(X_all, y_all)
print (score)
predict=clf2.predict(X_all)
df['predict']=predict
df.to_csv('c:\\temp\\all_result.csv')


0.9834494773519163


## Apply to New Dataset

In [10]:
dft=pd.read_csv('c:\\temp\\blah-testq4.csv')
dft

print (dft.columns)
len(dft['PART_DESC'])

###########**** make sure no empty end-row  *****########## 

Index(['Source Table', 'MACHINE_REGION', 'REF_NO', 'CUSTOMER', 'MACHINE_NO',
       'PARTS_REQUEST_REASON_CODE', 'APPLY_DATE', 'NEED_BY', 'SO_NO',
       'SO_LINE', 'PART_NO', 'PART_DESC', 'NEED_TO_BE_RETURNED', 'SHIPPED_QTY',
       'SHIPPING_ORG', 'SUBINVENTORY', 'DELIVERY_NUM', 'ACTUAL_SHIPPED_DATE',
       'APPROVED_DATE_2NDLINE', 'PRIORITY', 'FILLER', 'APPLICANT',
       'PARTS_REQUEST_STATUS', 'INV_AVAILABLE_DATE', 'INSTALLED_DATE',
       'NON_INSTALLED_REASON', 'NON_INSTALLED_RETURN', 'NON_INSTALL_DAYS',
       'INSTALLATION_STATUS', 'RMA_NO', 'RETURN_DATE', 'FORM_STATUS',
       'RETURNED_QTY', 'HMI_SN', 'Items_'],
      dtype='object')


292

In [9]:
Part_Desc='PART_DESC'
dft['PART NO_DESC']=dft[Part_Desc].str.strip()
dft['PART NO_DESC']=dft[Part_Desc].str.replace('-',",", regex=True)
dft['PART NO_DESC']=dft[Part_Desc].str.upper()


desc2=dft['PART NO_DESC'].values

X_N=vectorizer.transform(desc2)
predict2=clf2.predict(X_N)
predict2
dft['predict2']=predict2
dft.to_csv('c:\\temp\\predict.csv')


NameError: name 'dft' is not defined

In [17]:
import os
cwd=os.getcwd()
cwd

'C:\\Users\\suchung'